# PyDrill Demonstration
This notebook demonstrates how to use the PyDrill module to connect to Apache Drill and query data. The complete documentation for PyDrill can be found at http://pydrill.readthedocs.io

The essential steps are:
1.  Import the module
2.  Open a connection to Drill
3.  Execute a query
4.  Do something with the results. 

You will first need to install PyDrill.  This can be done by opening a terminal and typing:
```python
pip install pydrill
```
## Step 1:  Import the PyDrill module
After you've done this, you will be able to import the PyDrill module.

In [ ]:
from pydrill.client import PyDrill
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

## Step 2:  Open a connection to Drill
The next step is to open a connection to Drill.  Once you've opened the connection, you will want to verify that the connection was successfully opened before executing any queries.  PyDrill includes an `is_active()` method for this purpose.

In [ ]:
#Open a connection to Drill
drill = PyDrill(host='localhost', port=8047)

#Verify the connection is active, throw an error if not.
if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')

## Step 3: Execute a query and get the results
The next and final step is to execute a query in Drill.  When you call the `.query()` method, PyDrill returns an iterable object from which you can extract the rows of your results.  You can also get PyDrill to return a pandas DataFrame. 

In [ ]:
#Execute query in Drill
query_result = drill.query('''
SELECT JobTitle, 
AVG( TO_NUMBER( AnnualSalary, '¤' )) AS avg_salary, 
COUNT( DISTINCT `EmpName` ) AS number
FROM dfs.drillclass.`baltimore_salaries_2016.csvh`
GROUP BY JobTitle
Order BY avg_salary DESC
LIMIT 50
''')

#Iterate through the rows.
for row in query_result:
    print( row )

### Retrieving a DataFrame
You can also get PyDrill to directly return a DataFrame by using the `.to_dataframe()` method of the results object.

In [ ]:
df = query_result.to_dataframe()
df.head()

## In Class Exercise:
Using the data in the `dailybots.csv` file use Drill to:
1.  Query the file to produce a summary of infections by day.
2.  Store this data in a dataframe using the `to_dataframe()` method.
3.  Create a line plot of this data by calling the .plot() method on the dataframe

In order to render the plot, you will need to convert the column containing the numeric field into a numeric data type. The command below converts a column called `bot_count` to a floating point number which can be rendered in a line chart.
```python
df.bot_count = df.bot_count.astype(float)
```

If you are unfamiliar with pandas and the corresponding plotting libraries, the documentation is available here: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html.

In [ ]:
query_result = drill.query('''SELECT columns[0] AS bot_day, SUM(CAST( columns[3] AS INT )) AS bot_count
FROM dfs.drillclass.`dailybots.csv` 
WHERE NOT (columns[0] = 'date')
GROUP BY columns[0]''')

df1 = query_result.to_dataframe()
df1.bot_count = df1.bot_count.astype(float)

In [ ]:
df1.head()

In [ ]:
df1.plot( kind="line",
         x='bot_day',
         y='bot_count',
         figsize=(18,14)
        )